<h2> Featurizing text data with tfidf weighted word-vectors </h2>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm


import spacy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df = pd.read_csv("/content/drive/MyDrive/Project_gova/train.csv")

print(df.shape)
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

(101072, 6)


In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,325999,66488,63609,What are the safety precautions on handling sh...,What are the safety precautions on handling sh...,1
1,85974,145050,145051,Which type of question asked by Amazon intervi...,Should I co-author a book with my marketing pr...,0
2,277216,169290,6439,Where can I download free music?,How do I download free music?,1
3,88878,149407,149408,Why do some people hate Starbucks?,Why do people hate Starbucks?,1
4,166859,258801,258802,What is the most real life like movie about pi...,What are the most epic T-shirt quotes you have...,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))
len(word2tfidf)

57352

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics.

In [ ]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []

# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1)
    # print(len(doc1[0].vector))
    # 384 is the number of dimensions of vectors
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    # print(mean_vec1.shape)
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
    # print(vecs1)
df['q1_feats_m'] = list(vecs1)


100%|██████████| 101072/101072 [14:49<00:00, 113.58it/s]


In [ ]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2)
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████| 101072/101072 [14:46<00:00, 114.05it/s]


In [ ]:
!cp /content/drive/MyDrive/Project_gova/nlp_new_features_train.csv /content

In [ ]:
!cp /content/drive/MyDrive/Project_gova/df_new_without_preprocessing_train.csv /content

In [ ]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_new_features_train.csv'):
    dfnlp = pd.read_csv("nlp_new_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_new_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_new_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [ ]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [ ]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,325999,1,0.857131,0.857131,0.999983,0.999983,0.857137,0.857137,0.0,1.0,0.0,14.0,95,90,94,94,0.894118
1,85974,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.0,12.5,26,26,24,24,0.042254
2,277216,1,0.999967,0.999967,0.333322,0.333322,0.666656,0.666656,1.0,0.0,0.0,6.0,86,75,81,83,0.766667
3,88878,1,0.999967,0.999967,0.999950,0.666644,0.999980,0.833319,1.0,1.0,1.0,5.5,100,92,92,86,0.766667
4,166859,0,0.399992,0.249997,0.599988,0.374995,0.499995,0.312498,0.0,1.0,6.0,13.0,62,58,38,48,0.207547


In [ ]:
# data before preprocessing
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,325999,5,6,84,84,14,14,12.0,26.0,0.461538,11,1
1,85974,1,1,74,70,11,13,0.0,23.0,0.000000,2,0
2,277216,1,1,32,29,6,6,4.0,12.0,0.333333,2,0
3,88878,1,1,34,29,6,5,5.0,11.0,0.454545,2,0
4,166859,1,1,52,77,10,14,3.0,24.0,0.125000,2,0


In [ ]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,1.142547,20.798611,-1.911148,3.256480,28.333533,-5.536336,28.334607,-18.479668,35.815113,5.285083,...,24.327341,-15.612081,-31.799547,7.061352,-22.273014,9.430912,63.639465,51.716556,-0.968995,21.419110
1,40.751332,-25.022809,-13.782027,-0.156428,25.804808,-2.828593,33.068660,24.378593,-2.179438,-11.314334,...,-3.954990,-1.203053,-48.055441,28.722324,-15.668803,6.944211,59.573128,57.769425,-23.427030,41.253061
2,-1.010917,-5.997420,4.408809,2.891929,-6.104317,-11.931523,10.070111,5.593081,8.154651,-26.079729,...,0.235504,-4.889717,-5.656199,8.461195,3.497303,-10.274549,10.610606,13.471772,16.786932,16.101828
3,-22.471201,-8.423623,-1.206655,24.399410,0.701407,-17.026084,14.832584,20.337545,4.274443,-13.234160,...,-13.118156,-10.462898,-9.437935,0.243748,3.935030,4.828385,43.220259,2.292818,0.941497,21.457877
4,-1.469269,10.460877,0.655198,17.111741,15.159717,-23.034869,31.570416,13.530176,24.708706,-15.506909,...,-14.411526,16.195103,-0.496495,-10.111679,-10.091626,14.909663,65.259492,22.223920,-2.223895,2.970853


In [ ]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-0.797280,21.309287,-4.523283,1.339308,28.288394,-8.916272,28.385881,-14.896872,45.109152,2.311926,...,21.169828,-11.305044,-37.150649,6.308514,-27.080452,18.484972,67.264467,48.460133,1.327451,24.946161
1,18.246907,-37.859008,4.035011,15.594427,-32.544069,6.823254,20.060492,45.817067,-10.039178,-12.922974,...,18.843799,14.288198,-6.700126,29.148113,-23.321211,4.394194,8.594284,33.335841,-14.365679,47.014159
2,-10.599939,-6.011153,4.895726,1.009890,-5.955597,-13.727551,10.271176,3.597902,9.072536,-27.488784,...,5.097650,-3.538022,-9.421618,6.178239,5.271636,-8.024467,14.330006,17.037134,14.787957,18.480471
3,-23.565516,-10.777236,6.394736,17.340769,2.501633,-15.187066,14.771727,19.536404,2.459823,-14.548688,...,-20.446680,-9.133700,-4.813798,1.258837,4.231406,-2.350923,38.373792,5.716316,3.895628,25.379186
4,4.603554,-30.589839,1.538129,37.425724,-8.879522,-30.153560,25.020714,0.815904,2.462999,-5.624027,...,-4.984082,8.920693,-8.790442,23.307244,-5.725538,16.558474,88.171982,60.276696,1.099306,14.660433


In [ ]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [ ]:
# storing the final features to csv file
if not os.path.isfile('final_new_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_new_features.csv')

In [1]:
!cp /content/final_new_features.csv /content/drive/MyDrive/Project_gova

cp: cannot stat '/content/final_new_features.csv': No such file or directory
